In [2]:
# official_baseline.py (fixed)
# Decoder-only causal LM built with PyTorch's nn.TransformerEncoder.

import os, math, time, json, requests
from types import SimpleNamespace

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm

from models import tokenizer

# -----------------------------
# Load config
# -----------------------------
with open("config/hyperparameters.json", "r") as f:
    cfg = json.load(f, object_hook=lambda d: SimpleNamespace(**d))

data_path = cfg.data_path
save_path = cfg.save_path.replace(".pt", "_official.pt")  # avoid overwriting your original
split_ratio = tuple(cfg.split_ratio)
block_size = cfg.block_size
batch_size = cfg.batch_size
patience = cfg.patience
max_epochs = cfg.max_epochs
eval_interval_epochs = cfg.eval_interval
stride_overlap_ratio = cfg.stride_overlap_ratio

d_model = cfg.d_model
n_heads = cfg.n_heads
n_layers = cfg.n_layers
d_ff = cfg.d_ff
dropout = cfg.dropout
warmup = cfg.warmup

grad_clip = cfg.grad_clip
seed = cfg.seed

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [3]:
# -----------------------------
# Dataset fetch
# -----------------------------
os.makedirs("data", exist_ok=True)
if data_path == "data/tiny_shakespeare.txt":
    if os.path.exists(data_path):
        print(f"'{data_path}' already exists, skipping download.")
    else:
        url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
        text = requests.get(url).text
        with open("data/tiny_shakespeare.txt", "w", encoding="utf-8") as f:
            f.write(text)
        print("Tiny Shakespeare downloaded! File size:", len(text), "characters")
elif data_path == "data/full_shakespeare.txt":
    if os.path.exists(data_path):
        print(f"'{data_path}' already exists, skipping download.")
    else:
        url = "https://www.gutenberg.org/files/100/100-0.txt"
        print("Downloading full Shakespeare from Project Gutenberg...")
        text = requests.get(url).text
        if "*** START" in text:
            text = text.split("*** START")[1]
        if "*** END" in text:
            text = text.split("*** END")[0]
        with open(data_path, "w", encoding="utf-8") as f:
            f.write(text)
        print("Full Shakespeare downloaded! File size:", len(text), "characters")
else:
    raise SystemExit("Unexpected dataset, stop training.")

# -----------------------------
# Tokenizer & splits
# -----------------------------
with open(data_path, "r", encoding="utf-8") as f:
    text = f.read()

tok = tokenizer.CharTokenizer(text)
print(len(tok.chars), "unique chars")

ids = tok.encode(text)
data = torch.tensor(ids, dtype=torch.long)

vocab_size = getattr(tok, "vocab_size", len(tok.chars))
print("vocab_size =", vocab_size)
mx = int(max(ids)) if len(ids) > 0 else -1
assert mx < int(vocab_size), f"max id {mx} >= vocab_size {int(vocab_size)}"

n = len(data)
n_train = int(split_ratio[0] * n)
n_val = int(split_ratio[1] * n)
n_test = n - n_train - n_val

train_data = data[:n_train]
val_data = data[n_train:n_train + n_val]
test_data = data[n_train + n_val:]

print(f"Total tokens: {n:,}")
print(f"Train: {len(train_data):,}, Val: {len(val_data):,}, Test: {len(test_data):,}")

class CharDataset(torch.utils.data.Dataset):
    def __init__(self, data, block_size, stride=None):
        self.data = data
        self.block_size = block_size
        self.stride = self.block_size if (stride is None or stride < 1) else stride
        last_valid_start_idx = len(self.data) - self.block_size - 1
        self.num_samples = 0 if last_valid_start_idx < 0 else (last_valid_start_idx // self.stride) + 1

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        actual_idx = idx * self.stride
        x = self.data[actual_idx : actual_idx + self.block_size]
        y = self.data[actual_idx + 1 : actual_idx + 1 + self.block_size]
        return x, y

my_stride = int(block_size * stride_overlap_ratio)
train_dataset = CharDataset(train_data, block_size, stride=my_stride)
val_dataset   = CharDataset(val_data,   block_size, stride=my_stride)
test_dataset  = CharDataset(test_data,  block_size, stride=my_stride)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  pin_memory=True)
val_loader   = torch.utils.data.DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)
test_loader  = torch.utils.data.DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")


'data/full_shakespeare.txt' already exists, skipping download.
100 unique chars
vocab_size = 100
Total tokens: 5,359,388
Train: 4,287,510, Val: 535,938, Test: 535,940
Train batches: 1047, Val batches: 131


In [4]:
# -----------------------------
# Official-ish Decoder-only Transformer using nn.TransformerEncoder (causal mask)
# -----------------------------
class OfficialishCharLM(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, dropout, block_size):
        super().__init__()
        self.block_size = block_size
        self.d_model = d_model

        self.tok = nn.Embedding(vocab_size, d_model)
        self.pos = nn.Embedding(block_size, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads, dim_feedforward=d_ff,
            dropout=dropout, batch_first=True, norm_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.ln_f = nn.LayerNorm(d_model)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

        # Weight tying
        self.lm_head.weight = self.tok.weight

    def _causal_mask(self, T, device):
        # float mask with -inf on future positions
        return torch.triu(torch.ones(T, T, device=device) * float('-inf'), diagonal=1)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        if T > self.block_size:
            raise ValueError(f"Sequence length {T} exceeds block_size {self.block_size}")

        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)  # [1, T]
        x = self.tok(idx) * math.sqrt(self.d_model) + self.pos(pos)  # [B, T, C]

        causal = self._causal_mask(T, idx.device)  # [T, T]
        y = self.encoder(x, mask=causal)  # [B, T, C], encoder-only with causal mask
        y = self.ln_f(y)
        logits = self.lm_head(y)  # [B, T, V]

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.reshape(-1, logits.size(-1)), targets.reshape(-1))
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens=200, temperature=1.0, top_k=None):
        self.eval()
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)
            if top_k is not None:
                v, _ = torch.topk(logits, k=top_k)
                logits[logits < v[:, [-1]]] = -float("inf")
            probs = torch.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, next_id], dim=1)
        return idx

# -----------------------------
# Noam scheduler wrapper
# -----------------------------
class NoamOpt:
    def __init__(self, d_model, warmup, optimizer):
        self._step = 0
        self.warmup = warmup
        self.factor = d_model ** (-0.5)
        self.opt = optimizer
        self._lr = 0.0

    def step(self):
        self._step += 1
        lr = self.factor * min(self._step ** (-0.5), self._step * (self.warmup ** -1.5))
        for g in self.opt.param_groups:
            g['lr'] = lr
        self._lr = lr
        self.opt.step()

    @property
    def lr(self):
        return self._lr

    def zero_grad(self):
        self.opt.zero_grad(set_to_none=True)

In [5]:
def quick_lr_probe(warmups=(500, 1000, 2000)):
    probe = {}
    for wu in warmups:
        torch.manual_seed(seed)
        m = OfficialishCharLM(vocab_size, d_model, n_heads, n_layers, d_ff, dropout, block_size).to(device)
        base = torch.optim.Adam(m.parameters(), betas=(0.9,0.98), eps=1e-9)
        sched = NoamOpt(d_model, warmup=wu, optimizer=base)
        it = iter(train_loader)
        losses = []
        for _ in range(200):
            try:
                xb, yb = next(it)
            except StopIteration:
                it = iter(train_loader); xb, yb = next(it)
            xb, yb = xb.to(device), yb.to(device)
            _, loss = m(xb, yb)
            base.zero_grad(); loss.backward()
            torch.nn.utils.clip_grad_norm_(m.parameters(), grad_clip)
            sched.step()
            losses.append(loss.item())
        probe[wu] = sum(losses[-50:]) / 50 
    print("Probe val (lower is better):", probe)
    return probe

quick_lr_probe(warmups=(500, 1000, 2000))

d:\Anaconda\envs\Project_GenAI\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
d:\Anaconda\envs\Project_GenAI\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
d:\Anaconda\envs\Project_GenAI\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Probe val (lower is better): {500: 4.38028018951416, 1000: 6.238727722167969, 2000: 25.882893676757813}


{500: 4.38028018951416, 1000: 6.238727722167969, 2000: 25.882893676757813}

In [18]:
quick_lr_probe(warmups=(100, 200, 300))

Probe val (lower is better): {100: 3.479309668540955, 200: 3.6915267372131346, 300: 3.8168939924240113}


{100: 3.479309668540955, 200: 3.6915267372131346, 300: 3.8168939924240113}

In [6]:
quick_lr_probe(warmups=(10, 30, 50))

Probe val (lower is better): {10: 3.3596645832061767, 30: 3.370869073867798, 50: 4.400758528709412}


{10: 3.3596645832061767, 30: 3.370869073867798, 50: 4.400758528709412}

In [ ]:
# -----------------------------
# Model / Optimizer / Logger
# -----------------------------
torch.manual_seed(seed)
model = OfficialishCharLM(vocab_size, d_model, n_heads, n_layers, d_ff, dropout, block_size).to(device)
print("Params:", sum(p.numel() for p in model.parameters() if p.requires_grad))

run_dir = f"runs/official_{time.strftime('%Y%m%d-%H%M%S')}"
writer = SummaryWriter(log_dir=run_dir)
print("TensorBoard logdir:", run_dir)
os.makedirs(os.path.dirname(save_path), exist_ok=True)

base_opt = torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-9, weight_decay=0.0)
opt = NoamOpt(d_model, warmup=warmup, optimizer=base_opt)

best_val = float("inf")
bad_epochs = 0
global_step = 0

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        _, loss = model(xb, yb)
        total_loss += loss.item()
    model.train()
    return total_loss / max(len(loader), 1)

for epoch in range(1, max_epochs + 1):
    model.train()
    epoch_start_time = time.time()

    # Gradient accumulation (set >1 if want to simulate larger batches)
    accum_steps = 1

    # tqdm progress bar for the current epoch
    pbar = tqdm(
        enumerate(train_loader),
        total=len(train_loader),
        desc=f"Epoch {epoch}/{max_epochs}",
        dynamic_ncols=True,
        leave=False,  # keep only one dynamic line, no screen flooding
    )

    for batch_idx, (xb, yb) in pbar:
        xb, yb = xb.to(device), yb.to(device)

        # Forward + loss
        _, loss = model(xb, yb)
        loss = loss / accum_steps

        # Backward + gradient clipping + optimizer step
        opt.zero_grad()
        loss.backward()
        grad_norm = float(torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip))
        if (batch_idx + 1) % accum_steps == 0:
            opt.step()

        # Detach scalar values for display/logging
        loss_item = loss.item() * accum_steps
        bpc = loss_item / math.log(2)
        ppl = math.exp(loss_item) if loss_item < 20 else float("inf")

        # Update tqdm line with key metrics
        pbar.set_postfix({
            "loss": f"{loss_item:.4f}",
            "bpc": f"{bpc:.3f}",
            "ppl": f"{ppl:.1f}" if ppl != float("inf") else "inf",
            "lr": f"{opt.lr:.6f}",
            "gnorm": f"{grad_norm:.2f}",
        })

        # TensorBoard logging
        writer.add_scalar("loss/train_batch", loss_item, global_step)
        writer.add_scalar("lr", opt.lr, global_step)
        writer.add_scalar("grad_norm", grad_norm, global_step)
        global_step += 1

    # End of epoch summary
    # tqdm.write(f"[Epoch {epoch}] time={time.time() - epoch_start_time:.1f}s")

    # ---- Validation phase ----
    val_loss = evaluate(val_loader)
    bpc = val_loss / math.log(2)
    tqdm.write(f"[Epoch {epoch}] Val Loss {val_loss:.4f} | Val BPC {bpc:.4f} | time={time.time() - epoch_start_time:.1f}s")

    writer.add_scalar("loss/val", val_loss, epoch)
    writer.add_scalar("metrics/val_bpc", bpc, epoch)

    # ---- Early stopping and checkpointing ----
    if val_loss < best_val:
        best_val = val_loss
        bad_epochs = 0
        torch.save({
            "model": model.state_dict(),
            "optimizer": base_opt.state_dict(),
            "epoch": epoch,
            "best_val": best_val,
            "tok_chars": tok.chars,
            "config": dict(d_model=d_model, n_heads=n_heads, n_layers=n_layers, d_ff=d_ff,
                           dropout=dropout, block_size=block_size, vocab_size=vocab_size)
        }, save_path)
        tqdm.write(f"Improved! Best val_loss={best_val:.4f} (saved to {save_path})")
    else:
        bad_epochs += 1
        tqdm.write(f"No improvement ({bad_epochs}/{patience})")
        if bad_epochs >= patience:
            tqdm.write("Early stopping triggered.")
            break

In [ ]:
# -----------------------------
# Test best checkpoint + a sample generation
# -----------------------------
checkpoint = torch.load(save_path, map_location=device)
model.load_state_dict(checkpoint["model"])
model.to(device)
model.eval()

print(f"Loaded best model with best_val={checkpoint['best_val']:.4f} at epoch={checkpoint['epoch']}")

@torch.no_grad()
def evaluate_simple(loader):
    model.eval()
    total_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        _, loss = model(xb, yb)
        total_loss += loss.item()
    return total_loss / max(len(loader), 1)

test_loss = evaluate_simple(test_loader)
print(f"Final Test Loss: {test_loss:.4f}")
print(f"Perplexity (PPL): {torch.exp(torch.tensor(test_loss)):.2f}")
print(f"BPC: {(test_loss / math.log(2)):.4f}")

# Sample generation from a prompt
prompt = "ROMEO:"
start_ids = tok.encode(prompt)
idx = torch.tensor([start_ids], dtype=torch.long, device=device)
out = model.generate(idx, max_new_tokens=400, temperature=1.0, top_k=None)
print(tok.decode(out[0].tolist()))


In [ ]:
####### GRID SEARCH #######

In [27]:
import time, math, torch
from tqdm.auto import tqdm

# Run 1or3 Epoch for grid search
def run_experiment(d_model, n_heads, n_layers, d_ff, dropout, block_size, warmup,
                   batch_size=32, max_epochs=1): 
    torch.cuda.empty_cache()
    torch.manual_seed(42)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # ---- DataLoader rebuild ----
    train_dataset = CharDataset(train_data, block_size)
    val_dataset   = CharDataset(val_data, block_size)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    # ---- Model ----
    model = OfficialishCharLM(vocab_size, d_model, n_heads, n_layers, d_ff, dropout, block_size).to(device)
    base_opt = torch.optim.Adam(model.parameters(), betas=(0.9,0.98), eps=1e-9)
    opt = NoamOpt(d_model, warmup, base_opt)

    # ---- Training ----
    model.train()
    t0 = time.time()
    for xb, yb in tqdm(train_loader, total=len(train_loader),
                       desc=f"d_model={d_model}, n_layers={n_layers}, dropout={dropout}, block={block_size}"):
        xb, yb = xb.to(device), yb.to(device)
        _, loss = model(xb, yb)
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        opt.step()
    train_time = time.time() - t0

    # ---- Validation ----
    val_loss = 0.0
    with torch.no_grad():
        model.eval()
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            _, loss = model(xb, yb)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    bpc = val_loss / math.log(2)

    return dict(
        d_model=d_model, n_layers=n_layers, block_size=block_size,
        dropout=dropout, warmup=warmup, batch_size=batch_size,
        val_loss=val_loss, bpc=bpc, train_time=train_time
    )


In [28]:
import itertools
d_models = [128, 256, 384, 512]
n_layers = [4, 6, 8]
dropouts = [0.0, 0.1, 0.2]
block_sizes = [128, 256, 512]
warmup = 10

search_space = []
for d_model, n_layer, drop, block in itertools.product(d_models, n_layers, dropouts, block_sizes):
    n_heads = max(2, d_model // 64)  # keep roughly 64-dim per head
    d_ff = 4 * d_model
    search_space.append(dict(
        d_model=d_model, n_heads=n_heads, n_layers=n_layer,
        d_ff=d_ff, dropout=drop, block_size=block, warmup=warmup
    ))

print(f"Total configs: {len(search_space)}")


Total configs: 108


In [29]:
TARGET_TOKENS = 65536  # per optimizer step
def plan_bs(block):
    return max(1, TARGET_TOKENS // block)

for cfg in search_space:
    cfg["batch_size"] = plan_bs(cfg["block_size"])

In [30]:
results = []
for cfg in search_space:
    result = run_experiment(**cfg, max_epochs=3)
    results.append(result)

d_model=384, n_layers=6, dropout=0.0, block=128:  23%|██▎       | 15/66 [00:37<02:07,  2.50s/it]


KeyboardInterrupt: 

In [31]:
import pandas as pd
df = pd.DataFrame(results)
df = df.sort_values("bpc")
df

,d_model,n_layers,block_size,dropout,warmup,batch_size,val_loss,bpc,train_time
4,128,4,256,0.1,10,256,3.196448,4.611500,12.401019
3,128,4,128,0.1,10,512,3.212545,4.634723,11.949633
5,128,4,512,0.1,10,128,3.215670,4.639231,13.633289
51,256,8,128,0.2,10,512,3.218518,4.643340,327.032681
0,128,4,128,0.0,10,512,3.221088,4.647048,10.911317
...,...,...,...,...,...,...,...,...,...
44,256,6,512,0.2,10,128,3.457651,4.988335,120.000451
58,384,4,256,0.1,10,256,3.582868,5.168986,92.691968
55,384,4,256,0.0,10,256,3.623940,5.228241,47.270605
62,384,4,512,0.2,10,128,3.685264,5.316712,94.766493
